# 0. Install Dependencies

In [1]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2
!pip install pygame
!pip install nltk
!pip install requests

  Using cached scipy-1.4.1-cp37-cp37m-win_amd64.whl (30.9 MB)
  Using cached h5py-2.10.0-cp37-cp37m-win_amd64.whl (2.5 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.2.3
    Uninstalling scipy-1.2.3:
      Successfully uninstalled scipy-1.2.3
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0


ERROR: tensorflow-gpu 2.5.0 has requirement gast==0.4.0, but you'll have gast 0.3.3 which is incompatible.
ERROR: tensorflow-gpu 2.5.0 has requirement h5py~=3.1.0, but you'll have h5py 2.10.0 which is incompatible.
ERROR: tensorflow-gpu 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.18.5 which is incompatible.
ERROR: tensorflow-gpu 2.5.0 has requirement six~=1.15.0, but you'll have six 1.16.0 which is incompatible.
ERROR: tensorflow-gpu 2.5.0 has requirement tensorflow-estimator<2.6.0,>=2.5.0rc0, but you'll have tensorflow-estimator 2.3.0 which is incompatible.
ERROR: gym 0.18.3 has requirement Pillow<=8.2.0, but you'll have pillow 8.3.1 which is incompatible.


  Attempting uninstall: Pillow
    Found existing installation: Pillow 8.3.1
    Uninstalling Pillow-8.3.1:
      Successfully uninstalled Pillow-8.3.1


# 1. Test Random Environment with OpenAI Gym

In [2]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random
import pygame
import os
import nltk
import string
import requests

C:\Users\rodam\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\rodam\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\rodam\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


pygame 2.0.1 (SDL 2.0.14, Python 3.7.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
window_width, window_height = 1000, 500

# api data
data_api = requests.get("https://api.bridgedataoutput.com/api/v2/OData/test/Property?access_token=b9fb97d623d73441e25ee82bfcb8c4fa")

In [ ]:
class RealEstateEnv(Env):
    def __init__(self, houses):
        # Actions we can take, lower price, same price, increase price
        self.action_space = Discrete(3)
        # House prices array
        self.observation_space = Box(low=np.array([100000]), high=np.array([1100000]))
        # Set state
        self.state = 0
        # Set finding length
        self.finding_length = 180
        
        self.pos = random.randint(0, len(houses)-1)
        self.house = houses[self.pos]
    
    def init_render(self):
        pygame.init()
        self.window = pygame.display.set_mode((window_width, window_height))
        pygame.display.set_caption("Real Estate Env")
        self.clock = pygame.time.Clock()
    
    
    def text(self, text):
        font_color=(0,150,250)
        font_obj=pygame.font.Font(None, 25)
        # Render the objects
        text_obj = []
        for t in text:
            text_obj.append(font_obj.render(t, True, font_color))
        return text_obj
        
    def step(self, action, customer, houses):
        
        
        # Apply action
        if action == 0 and self.pos != 0:
            self.pos -= 1
        elif action == 2 and self.pos != len(houses)-1:
            self.pos += 1
        
        self.house = houses[self.pos]
        
        bedroom_dif = (self.house.bedrooms / customer.num_bedrooms) * (1/3)
        square_footage_dif = (self.house.square_footage / customer.square_footage) * (1/3)
        price_dif = (self.house.price / customer.price) * (1/3)
        
        state2 = bedroom_dif + square_footage_dif + price_dif
        
        # Reduce house finding length by 1 second
        self.finding_length -= 1
        
        # Calculate reward
        if abs(1 - self.state) > abs(1 - state2): 
            self.state = state2
            reward = 1
        else:
            reward = -1
        
        # Check if finding time is over
        if self.finding_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply price fluctuations
        # self.state += random.randint(-10000,25000)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self, text_obj):
        # Implement viz
        self.window.fill((0,0,0))
        # draw orientation
        for i in range(0, len(text_obj)):
            self.window.blit(text_obj[i],(22,i*50))
        #left person
        #head
        pygame.draw.circle(self.window, (66, 84, 81), (int(150), int(320)), 40)
        #eyes
        pygame.draw.circle(self.window, (0, 200, 200), (int(130), int(310)), 8)
        pygame.draw.circle(self.window, (0, 200, 200), (int(170), int(310)), 8)
        #body
        pygame.draw.rect(self.window, (45,45,45), [120, 360, 60, 140])
        #right person
        #head
        pygame.draw.circle(self.window, (250,227,130), (window_width-int(150), int(320)), 40)
        #eyes
        pygame.draw.circle(self.window, (0, 20, 20), (window_width-int(130), int(310)), 8)
        pygame.draw.circle(self.window, (0, 20, 20), (window_width-int(170), int(310)), 8)
        #mouth
        pygame.draw.line(self.window, (255, 205, 205), [window_width-int(135), int(338)],[window_width-int(165),int(338)], 6)
        #body
        pygame.draw.rect(self.window, (220,197,100), [window_width-180, 360, 60, 140])
        pygame.display.update()
    
    def reset(self):
        # Reset base price
        self.state = 0
        # Reset finding time
        self.finding_length = 180
        return self.state

In [ ]:
class Customer():
    def __init__(self, num):
        self.square_footage = random.randint(500, 4500)
        self.num_bedrooms = round(self.square_footage / 800)
        self.price = self.square_footage * 500
        self.id = num
    
    @staticmethod
    def load_customers(x):
        arr = []
        
        for i in range(x):
            arr.append(Customer(i + 1))
        
        return arr

In [ ]:
class Houses():
    def __init__(self, properties):
        self.square_footage = int(properties[0])
        self.bedrooms = int(properties[1])
        self.price = int(properties[2])

In [ ]:
data_dir = "./data"
num_properties = 3
with open(data_dir) as f:
    s = f.read()

word_list = nltk.word_tokenize(s)

for word in word_list:
    if not word.isdigit():
        word_list.remove(word)

houses = []
temp2 = []
for i in range(len(word_list)):
    temp2.append(word_list[i])
    if len(temp2) == 3:
        house = Houses(temp2)
        houses.append(house)
        temp2.clear()

houses = sorted(houses, key=lambda x: x.price)

num = 0
for house in houses:
    print("House " + str(num))
    print("Square footage: {} Bedrooms: {} Price: {}".format(house.square_footage, house.bedrooms, house.price))
    num += 1
    print()

In [ ]:
env = RealEstateEnv(houses)
env.init_render()

In [ ]:
env.observation_space.sample()

In [ ]:
episodes = 10
num_customers = 5
customers = Customer.load_customers(num_customers)
pygame.init()
for i in range(num_customers):
    for episode in range(1, episodes+1):
        state = env.reset()
        done = False
        score = 0 

        while not done:
            textArray = ["Score: " + str(env.state),
                         "Current Price: " + str(env.house.price), 
                         "Goal Price: " + str(customers[i].price), 
                         "Current footage: " + str(env.house.square_footage), 
                         "Goal Footage: " + str(customers[i].square_footage), 
                         "Current Bedrooms: " + str(env.house.bedrooms), 
                         "Goal Bedrooms: " + str(customers[i].num_bedrooms)]
            text_obj = env.text(textArray)
            env.clock.tick(30)
            action = env.action_space.sample()
            n_state, reward, done, info = env.step(action, customers[i], houses)
            score+=reward
            env.render(text_obj)
            for eve in pygame.event.get():
                if eve.type==pygame.QUIT:
                    pygame.quit()
                    #sys.exit()
        print('Customer ID: {} Episode:{} Score:{}'.format(customers[i].id, episode, score))

pygame.quit()

# 2. Create a Deep Learning Model with Keras

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
states = env.observation_space.shape
actions = env.action_space.n

In [ ]:
actions

In [ ]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [ ]:
del model 

In [ ]:
model = build_model(states, actions)

In [ ]:
model.summary()

# 3. Build Agent with Keras-RL

In [ ]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [ ]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

In [ ]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

In [ ]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

# 4. Reloading Agent from Memory

In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [ ]:
del model
del dqn
del env

In [ ]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
dqn.load_weights('dqn_weights.h5f')

In [ ]:
_ = dqn.test(env, nb_episodes=5, visualize=True)